In [1]:
from src.load import load_data

%load_ext autoreload
%autoreload 2

In [2]:
X, X_obs, y, X_test, X_test_obs, features = load_data()

Loading data...
Adding basic features...
Encoding data...
Removing extreme values... (> 3 ticks)
    [Train set] Filtering out of range prices: 7810706 rows removed over 16080000 rows (48.57%)
    [Test set] Filtering out of range prices: 3905944 rows removed over 8160000 rows (47.87%)
Normalizing data...(SKIPPED)
Converting to numpy arrays...


In [3]:
X_obs

array([0, 0, 0, ..., 160799, 160799, 160799], dtype=object)

In [42]:
import torch
import os
from tqdm import tqdm
import numpy as np
import networkx as nx
from torch_geometric.utils.convert import from_scipy_sparse_matrix

c:\Users\ezzeh\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
index = np.array(list(set(X_obs)))
split = "train"

In [ ]:
for i in tqdm(index, desc="Processing data", leave=False):
    if split == "train" or split == "val":
        label = torch.tensor(y[i], dtype=torch.long)
    else:
        label = torch.tensor(-1, dtype=torch.long)

In [ ]:
i = 0

graph = nx.Graph()

In [18]:
features

array(['venue', 'order_id', 'action', 'side', 'price', 'bid', 'ask',
       'bid_size', 'ask_size', 'trade', 'flux', 'bid_ask_spread',
       'Limit Order'], dtype=object)

In [24]:
node_features = np.array(["bid_ask_spread", "side", "price", "bid_size", "ask_size", "venue", "flux"])

In [26]:
# indices where the node features are located in the feature vector
node_features_indices = [np.where(features == node_feature)[0][0] for node_feature in node_features]
node_features_indices

[11, 3, 4, 7, 8, 0, 10]

In [29]:
X.shape

(8269294, 13)

In [31]:
i = 0

graph = nx.Graph()
node_features = ["bid_ask_spread", "side", "price", "bid_size", "ask_size", "venue", "flux"]
node_features_indices = [np.where(features == node_feature)[0][0] for node_feature in node_features]

curr_X = X[X_obs == i]
node_arr = torch.zeros((len(curr_X), len(node_features)), dtype=torch.float32)
venues_previous_keys = {}
order_ids_keys = {}
for k, row in enumerate(curr_X):
    order_id = row[1]
    venue = row[0]

    graph.add_node(k)
    node_arr[k, :] = torch.tensor(
        [row[j] for j in node_features_indices],
        dtype=torch.float32,
    )

In [43]:
i = 0

graph = nx.Graph()
node_features = np.array(["bid_ask_spread", "side", "price", "bid_size", "ask_size", "venue", "flux"])
node_features_indices = [np.where(features == node_feature)[0][0] for node_feature in node_features]

edge_temp_features = np.array(["trade", "Limit Order"])
edge_temp_features_indices = [np.where(features == edge_feature)[0][0] for edge_feature in edge_temp_features]

edge_id_features = np.array(["trade", "Limit Order"])
edge_id_features_indices = [np.where(features == edge_feature)[0][0] for edge_feature in edge_id_features]

curr_X = X[X_obs == i]
node_arr = torch.zeros((len(curr_X), len(node_features)), dtype=torch.float32)
venues_previous_keys = {}
order_ids_keys = {}
for k, row in enumerate(curr_X):
    order_id = row[1]
    venue = row[0]

    graph.add_node(k)
    node_arr[k, :] = torch.tensor(
        [row[j] for j in node_features_indices],
        dtype=torch.float32,
    )
    temp_attr = torch.tensor(
        [row[j] for j in edge_temp_features_indices],
        dtype=torch.int,
    )
    graph.add_edge(k, k-1, edge_attr=temp_attr) # Temporal edge

    id_attr = torch.tensor(
        [row[j] for j in edge_id_features_indices],
        dtype=torch.int,
    )

    if order_id in order_ids_keys:
        graph.add_edge(k, order_ids_keys[order_id][-1], edge_attr=id_attr)
        order_ids_keys[order_id].append(k)
    else:
        order_ids_keys[order_id] = [k]

edge_index, _ = from_scipy_sparse_matrix(nx.adjacency_matrix(graph))

In [ ]:
for i in tqdm(index, desc="Processing data", leave=False):
    if split == "train" or split == "val":
        label = torch.tensor(y[i], dtype=torch.long)
    else:
        label = torch.tensor(-1, dtype=torch.long)

    graph = nx.Graph()
    node_features = torch.zeros((len(X[i]), 7), dtype=torch.float32)
    venues_previous_keys = {}
    order_ids_keys = {}
    for k, row in enumerate(X[i]):
        order_id = row[1]
        venue = row[0]

        graph.add_node(k)
        node_features[k, :] = torch.tensor(
            [venue, k, row[4], row[6] - row[5], row[7], row[8], row[10]],
            dtype=torch.float32,
        )
        attr = torch.tensor(
            [row[2] == 0, row[2] == 1, row[2] == 2, row[3], row[9]],
            dtype=torch.int,
        ) # edge features

        if venue in venues_previous_keys:
            graph.add_edge(k, venues_previous_keys[venue], edge_attr=attr)
        venues_previous_keys[venue] = k

        if order_id in order_ids_keys:
            graph.add_edge(k, order_ids_keys[order_id][-1], edge_attr=attr)
            order_ids_keys[order_id].append(k)
        else:
            order_ids_keys[order_id] = [k]

    edge_index, _ = from_scipy_sparse_matrix(nx.adjacency_matrix(graph))

    torch.save(
        Data(
            edge_index=edge_index,
            x=node_features,
            edge_attr=torch.stack(
                list(nx.get_edge_attributes(graph, "edge_attr").values())
            ).repeat_interleave(2, dim=0),
            y=label,
        ),
        os.path.join(self.processed_dir, f"{i}.pt"),
    )
